<a href="https://colab.research.google.com/github/divaamahajan/Spark-Logs/blob/kafkaLogs/kafkaProducer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up Dependencies

In [ ]:
!pip install confluent-kafka==2.1.1
!pip install kafka==1.3.5
!pip install kafka-python==2.0.2
!pip install numpy==1.24.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 54.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement pkg_resources==0.0.0 (from versions: none)
ERROR: No matching distribution found for pkg_resources==0.0.0


In [ ]:
!pip install pkg_resources==0.0.0
!pip install py4j==0.10.9.7
!pip install pyspark==3.4.0
!pip install python-dateutil==2.8.2
!pip install pytz==2023.3
!pip install six==1.16.0
!pip install spark==0.2.1
!pip install tzdata==2023.3
!apt-get install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pkg_resources==0.0.0 (from versions: none)
ERROR: No matching distribution found for pkg_resources==0.0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=0e31bf55dd18e5d197038ab1f3798d6b324441b87eef08fc72bad76b3d59e85d
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes:

In [ ]:
!pip install confluent_kafka sseclient kafka

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Using cached kafka-1.3.5-py2.py3-none-any.whl (207 kB)
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5565 sha256=61d6dd5079fa7cf3cf90859aaa5a0175652d97d49dd4f0fa5e4b0cf66c0ff856
  Stored in directory: /root/.cache/pip/wheels/60/57/0e/09b1264923280e935a34cc543b7f147f5df12490bd7a992f42
Successfully built sseclient


In [ ]:
!pip uninstall kafka
!pip uninstall kafka-python
!pip install kafka-python

Found existing installation: kafka 1.3.5
Uninstalling kafka-1.3.5:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/kafka-1.3.5.dist-info/*
    /usr/local/lib/python3.10/dist-packages/kafka/*
Proceed (Y/n)? Y
  Successfully uninstalled kafka-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)


In [3]:
import zipfile
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from confluent_kafka import Producer, Consumer, KafkaError



## Set up Topic and broker

In [12]:
# Specify the Kafka broker and topic to consume from
brokers = 'localhost:9092'
topic = 'log_lines_topic'
parquet_host = 'localhost:9000'


## 1. Extract the zip file

In [13]:
dataset = '42MBSmallServerLog.log'
zip_file_path = f'{dataset}.zip'
extracted_folder_path = 'extracted_dataset'

In [14]:

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)


## 2. Set up the Kafka producer

In [15]:
from confluent_kafka import Producer
producer = Producer({
    'bootstrap.servers': brokers,  # Kafka broker address
    'api.version.request': True
})

print('Kafka Producer has been initiated...')

Kafka Producer has been initiated...


## Read log File

In [16]:
def receipt(err,msg):
    if err is not None:
        print('Error: {}'.format(err))

In [17]:
# Read log file
log_file_path = f'{extracted_folder_path}/{dataset}'
batch_size = 1000  # Adjust this value based on requirements
# Open the log file
with open(log_file_path, 'r') as file:
    batch = []
    for line in file:
        # Add each line to the current batch
        batch.append(line)
        # Check if the batch size is reached
        if len(batch) >= batch_size:
            # Produce the batch to Kafka
            # producer.send(topic, value=''.join(batch).encode('utf-8'))
            producer.produce(topic, value=''.join(batch).encode('utf-8'))
            # producer.flush()
            # Clear the batch
            batch = []
    # Check if there are any remaining lines in the last batch
    if batch:
        # Produce the remaining lines to Kafka
        producer.poll(1)
        producer.produce(topic, value=''.join(batch).encode('utf-8'),callback=receipt)
        # producer.flush()
# Flush any remaining messages in the producer queue
producer.flush()
print("Data flushed to Producer Queues")

Data flushed to Producer Queues
